In [1]:
#
# a program to make .fsgd file for fREEsURFER-script fs_glm.sh from .xlsx files
#
import pandas as pd
import os
IMPACT_CWD=os.path.expanduser("~")+"/Downloads/impact/" # working directory
#print(IMPACT_CWD)
bl_book = pd.ExcelFile(IMPACT_CWD+'BHQ_Research_2018_01_Access用_20200108.xlsx') # base lien file
fu_book = pd.ExcelFile(IMPACT_CWD+'BHQ_Research_2018_02_Access用_20200108.xlsx') # follow up file

# 手動で取り除くIDを指定
IDsQCfailed=["MR0011_0742","MR0011_0770","MR0011_0810"] # QCでNGのID
IDsQCfailed=IDsQCfailed+["MR0011_0681","MR0011_0787","MR0011_0793"] # niftiファイルの無いID
IDsQCfailed=IDsQCfailed+["MR0011_0761"] # follow up画像の無いID
print("取り除くID:",IDsQCfailed)

取り除くID: ['MR0011_0742', 'MR0011_0770', 'MR0011_0810', 'MR0011_0681', 'MR0011_0787', 'MR0011_0793', 'MR0011_0761']


In [2]:
# read base line
bl_sheet_name = bl_book.sheet_names
bl_sheet_df = bl_book.parse(bl_sheet_name[0],skiprows=3,skipfooter=5)
bl_sheet_df=bl_sheet_df[bl_sheet_df[bl_sheet_df.columns[1]].str.contains("",na=False)] # id exists
#print("Sheet name:", bl_sheet_name[0])
bl_sheet_df=bl_sheet_df.loc[:,["ImPACT_ID","Arm_Type_01","Age_01","Gender_12_01","MRI_Imaging_Date_01",\
                               "FABHQ_01","FABHQ_DMN_01","FABHQ_CEN_01","FABHQ_SN_01",\
                               "GMBHQ_01","GMBHQ_DMN_01","GMBHQ_CEN_01","GMBHQ_SN_01",\
                               "SPA_Unrelated_SUM_01","TMT_PartA_01_01","TMT_PartB_01_01",\
                               "MMSEJ_SUM_01","CESD_Score_01"]] # only base lines have MMSE
#print(bl_sheet_df.shape)
#print(bl_sheet_df.tail(5))

# read follow up
fu_sheet_name = fu_book.sheet_names
fu_sheet_df=fu_book.parse(fu_sheet_name[0],skiprows=3,skipfooter=5)
fu_sheet_df=fu_sheet_df[fu_sheet_df[fu_sheet_df.columns[1]].str.contains("",na=False)] # id exists
#print("Sheet name:", fu_sheet_name[0])
fu_sheet_df=fu_sheet_df.loc[:,["ImPACT_ID","Arm_Type_02","Age_02","Gender_12_02","MRI_Imaging_Date_02",\
                               "FABHQ_02","FABHQ_DMN_02","FABHQ_CEN_02","FABHQ_SN_02",\
                               "GMBHQ_02","GMBHQ_DMN_02","GMBHQ_CEN_02","GMBHQ_SN_02",\
                               "SPA_Unrelated_SUM_01","TMT_PartA_01_02","TMT_PartB_01_02"]] #,"MMSEJ_SUM_01","CESD_Score_01"
#print(fu_sheet_df.shape)
#print(fu_sheet_df.tail(5))

# merge baseline and followup
bl_fu_sheet_df=pd.merge(bl_sheet_df, fu_sheet_df, left_on=bl_sheet_df.columns[0], right_on=fu_sheet_df.columns[0], how="inner", suffixes = ["_bl", "_fu"])
bl_fu_sheet_df.columns=bl_fu_sheet_df.columns.str.replace(" ","_")
bl_fu_sheet_df.columns=bl_fu_sheet_df.columns.str.replace("\n","_")

In [3]:
# remove IDs whose QC failed
bl_fu_sheet_df=bl_fu_sheet_df[~bl_fu_sheet_df["ImPACT_ID"].isin(IDsQCfailed)]

# remove IDs depressed
#print("IDs depressed:\n",bl_fu_sheet_df[bl_fu_sheet_df["CESD_Score_01"]>=16][["ImPACT_ID","Arm_Type_01","CESD_Score_01"]])
bl_fu_sheet_df=bl_fu_sheet_df[bl_fu_sheet_df["CESD_Score_01"]<16]

# make data frame
df_result=bl_fu_sheet_df.loc[:,["ImPACT_ID","Arm_Type_01","Age_01","Gender_12_01","MRI_Imaging_Date_01",\
                               "SPA_Unrelated_SUM_01_bl","TMT_PartA_01_01","TMT_PartB_01_01",\
                               "MMSEJ_SUM_01",\
                                          "Age_02","MRI_Imaging_Date_02",\
                               "SPA_Unrelated_SUM_01_fu","TMT_PartA_01_02","TMT_PartB_01_02"]]
df_result["fID"]="V_"+df_result["ImPACT_ID"]+"_"+df_result["Arm_Type_01"]
#print(df_result.shape)
#print(df_result.head(2))
#print(type(df_result))
#df_result=df_result.drop(columns=["Age_02"]) # work around for coincidence
#df_result=df_result.T.drop_duplicates().T # 重複列削除
df_result=df_result.drop(columns=["MRI_Imaging_Date_01", "MRI_Imaging_Date_02"])
df_result=df_result.set_index("fID")
#print(df_result.shape)
#print(df_result.head(2))
#print(type(df_result))

In [4]:
# make .fsgd file for fREEsURFER-script fs_glm.sh
ctrl_grouplist=["AR"]
intrv_grouplist=["AS","AT","AU"]
#impact_grouplist=["AR","AS","AT","AU"]
suffixlist=["_01","_02"]
scorelist=["SPA_Unrelated_SUM_01_bl","TMT_PartA_01_01","TMT_PartB_01_01","MMSEJ_SUM_01",\
           "SPA_Unrelated_SUM_01_fu","TMT_PartA_01_02","TMT_PartB_01_02"]

In [5]:
# g2v0: group別の皮質厚の差
temp_title="group_"
for sfx in suffixlist:
    for ss in intrv_grouplist:
        classlist=[ctrl_grouplist[0], ss]
        filename=IMPACT_CWD+"g2v0_"+ss+sfx+".fsgd"
        outfile=open(filename, "wt")
        outfile.write("GroupDescriptorFile 1"+"\n")
        outfile.write("Title"+" "+temp_title+ss+sfx+"\n")
        outfile.write("MeasurementName thickness"+"\n")
        outfile.write("Class "+classlist[0]+"\n")
        outfile.write("Class "+classlist[1]+"\n")
        for temp_line in df_result.itertuples():
            temp_armtype=temp_line[df_result.columns.get_loc("Arm_Type_01")+1] # 1 for .index
            if temp_armtype in classlist:
                outfile.write("Input"+" "+temp_line.Index+sfx+" "+temp_armtype+"\n")
        outfile.close()

filename=IMPACT_CWD+"g2v0_common.mtx"
outfile=open(filename, "wt")
outfile.write(str(1)+" "+str(-1))
outfile.close()
#
# usage:
# from Terminal$ call fREEsURFER-scripts as follows
# mkdir $SUBJECTS_DIR/glm may be necessary
#
"""
cp *.mtx $SUBJECTS_DIR/fsaverage/surf
cp *.fsgd $SUBJECTS_DIR/fsaverage/surf
cd $SUBJECTS_DIR/fsaverage/surf
temp_fsgd=g2v0_
temp_title=group_
temp_sfxlist="_01 _02"
temp_classlist="AS AT AU"
temp_hemilist="lh rh"
temp_mtx=g2v0_common
for sfxlist in ${temp_sfxlist};do
    for classlist in ${temp_classlist};do 
        #echo fs_glm.sh ${temp_fsgd}${classlist}${sfxlist}.fsgd ${temp_mtx}.mtx;
        yes| fs_glm.sh ${temp_fsgd}${classlist}${sfxlist}.fsgd ${temp_mtx}.mtx;
    done;
done
for sfxlist in ${temp_sfxlist};do
    for classlist in ${temp_classlist};do 
        for hemilist in ${temp_hemilist};do
            #echo fs_view_glm.sh ${hemilist}.${temp_title}${classlist}${sfxlist}.thickness.10.glmdir ${temp_mtx}.mtx 1 4;
            yes| fs_view_glm_allfwhm.sh ${hemilist}.${temp_title}${classlist}${sfxlist}.thickness.10.glmdir ${temp_mtx}.mtx 1 4;
        done;
    done;
done
# 多重比較補正 CDT=1.3(p＝0.05)は不適。実際にはCDT=3.0(p=0.001)で行えば良い。
for sfxlist in ${temp_sfxlist};do
    for classlist in ${temp_classlist};do 
        for hemilist in ${temp_hemilist};do
            #echo fs_glmfit-sim.sh $SUBJECTS_DIR/fsaverage/surf/${hemilist}.${temp_title}${classlist}${sfxlist}.thickness.10.glmdir 1.3;
            yes| fs_glmfit-sim.sh $SUBJECTS_DIR/fsaverage/surf/${hemilist}.${temp_title}${classlist}${sfxlist}.thickness.10.glmdir 1.3;
            #yes| fs_glmfit-sim.sh $SUBJECTS_DIR/fsaverage/surf/${hemilist}.${temp_title}${classlist}${sfxlist}.thickness.10.glmdir 3.0;
        done;
    done;
done
for sfxlist in ${temp_sfxlist};do
    for classlist in ${temp_classlist};do 
        for hemilist in ${temp_hemilist};do
            #echo freeview -f ${hemilist}.inflated:overlay=${hemilist}.${temp_title}${classlist}${sfxlist}.thickness.10.glmdir/${temp_mtx}/cache.th13.pos.sig.cluster.mgh:annot=${hemilist}.${temp_title}${classlist}${sfxlist}.thickness.10.glmdir/${temp_mtx}/cache.th13.pos.sig.ocn.annot -layout 1 -viewport 3d;
            freeview -f ${hemilist}.inflated:overlay=${hemilist}.${temp_title}${classlist}${sfxlist}.thickness.10.glmdir/${temp_mtx}/cache.th13.pos.sig.cluster.mgh:annot=${hemilist}.${temp_title}${classlist}${sfxlist}.thickness.10.glmdir/${temp_mtx}/cache.th13.pos.sig.ocn.annot -layout 1 -viewport 3d;
            #freeview -f ${hemilist}.inflated:overlay=${hemilist}.${temp_title}${classlist}${sfxlist}.thickness.10.glmdir/${temp_mtx}/cache.th30.pos.sig.cluster.mgh:annot=${hemilist}.${temp_title}${classlist}${sfxlist}.thickness.10.glmdir/${temp_mtx}/cache.th30.pos.sig.ocn.annot -layout 1 -viewport 3d;
        done;
    done;
done
"""

'\ncp *.mtx $SUBJECTS_DIR/fsaverage/surf\ncp *.fsgd $SUBJECTS_DIR/fsaverage/surf\ncd $SUBJECTS_DIR/fsaverage/surf\ntemp_fsgd=g2v0_\ntemp_title=group_\ntemp_sfxlist="_01 _02"\ntemp_classlist="AS AT AU"\ntemp_hemilist="lh rh"\ntemp_mtx=g2v0_common\nfor sfxlist in ${temp_sfxlist};do\n    for classlist in ${temp_classlist};do \n        #echo fs_glm.sh ${temp_fsgd}${classlist}${sfxlist}.fsgd ${temp_mtx}.mtx;\n        yes| fs_glm.sh ${temp_fsgd}${classlist}${sfxlist}.fsgd ${temp_mtx}.mtx;\n    done;\ndone\nfor sfxlist in ${temp_sfxlist};do\n    for classlist in ${temp_classlist};do \n        for hemilist in ${temp_hemilist};do\n            #echo fs_view_glm.sh ${hemilist}.${temp_title}${classlist}${sfxlist}.thickness.10.glmdir ${temp_mtx}.mtx 1 4;\n            yes| fs_view_glm_allfwhm.sh ${hemilist}.${temp_title}${classlist}${sfxlist}.thickness.10.glmdir ${temp_mtx}.mtx 1 4;\n        done;\n    done;\ndone\n# 多重比較補正 CDT=1.3(p＝0.05)は不適。実際にはCDT=3.0(p=0.001)で行えば良い。\nfor sfxlist in ${temp_sfxlis

In [6]:
# g2v1: 年齢調整後のgroup別の皮質厚の差
# g2v1_gx: group別で年齢と皮質厚の相関に違いがある領域
temp_title="group_age_"
for sfx in suffixlist:
    for ss in intrv_grouplist:
        classlist=[ctrl_grouplist[0], ss]
        filename=IMPACT_CWD+"g2v1_"+ss+sfx+".fsgd"
        outfile=open(filename, "wt")
        outfile.write("GroupDescriptorFile 1"+"\n")
        outfile.write("Title"+" "+temp_title+ss+sfx+"\n")
        outfile.write("MeasurementName thickness"+"\n")
        outfile.write("Class "+classlist[0]+"\n")
        outfile.write("Class "+classlist[1]+"\n")
        outfile.write("Variables"+" "+"age"+"\n")
        for temp_line in df_result.itertuples():
            temp_armtype=temp_line[df_result.columns.get_loc("Arm_Type_01")+1] # 1 for .index
            temp_age=temp_line[df_result.columns.get_loc("Age"+sfx)+1] # 1 for .index
            if temp_armtype in classlist:
                outfile.write("Input"+" "+temp_line.Index+sfx+" "+temp_armtype+" "+str(temp_age)+"\n")
        outfile.close()

filename=IMPACT_CWD+"g2v1_common.mtx"
outfile=open(filename, "wt")
outfile.write(str(1)+" "+str(-1)+" "+str(0)+" "+str(0))
outfile.close()

filename=IMPACT_CWD+"g2v1_gx_common.mtx"
outfile=open(filename, "wt")
outfile.write(str(0)+" "+str(0)+" "+str(1)+" "+str(-1))
outfile.close()
#
# usage:
# from Terminal$ call fREEsURFER-scripts as follows
# mkdir $SUBJECTS_DIR/glm may be necessary
#
"""
cp *.mtx $SUBJECTS_DIR/fsaverage/surf
cp *.fsgd $SUBJECTS_DIR/fsaverage/surf
cd $SUBJECTS_DIR/fsaverage/surf
temp_fsgd=g2v1_
temp_title=group_age_
temp_sfxlist="_01 _02"
temp_classlist="AS AT AU"
temp_hemilist="lh rh"
temp_mtx=g2v1_common
#temp_mtx=g2v1_gx_common
for sfxlist in ${temp_sfxlist};do
    for classlist in ${temp_classlist};do 
        #echo fs_glm.sh ${temp_fsgd}${classlist}${sfxlist}.fsgd ${temp_mtx}.mtx;
        yes| fs_glm.sh ${temp_fsgd}${classlist}${sfxlist}.fsgd ${temp_mtx}.mtx;
    done;
done
for sfxlist in ${temp_sfxlist};do
    for classlist in ${temp_classlist};do 
        for hemilist in ${temp_hemilist};do
            #echo fs_view_glm.sh ${hemilist}.${temp_title}${classlist}${sfxlist}.thickness.10.glmdir ${temp_mtx}.mtx 1 4;
            yes| fs_view_glm_allfwhm.sh ${hemilist}.${temp_title}${classlist}${sfxlist}.thickness.10.glmdir ${temp_mtx}.mtx 1 4;
        done;
    done;
done
# 多重比較補正 CDT=1.3(p＝0.05)は不適。実際にはCDT=3.0(p=0.001)で行えば良い。
for sfxlist in ${temp_sfxlist};do
    for classlist in ${temp_classlist};do 
        for hemilist in ${temp_hemilist};do
            #echo fs_glmfit-sim.sh $SUBJECTS_DIR/fsaverage/surf/${hemilist}.${temp_title}${classlist}${sfxlist}.thickness.10.glmdir 1.3;
            yes| fs_glmfit-sim.sh $SUBJECTS_DIR/fsaverage/surf/${hemilist}.${temp_title}${classlist}${sfxlist}.thickness.10.glmdir 1.3;
            #yes| fs_glmfit-sim.sh $SUBJECTS_DIR/fsaverage/surf/${hemilist}.${temp_title}${classlist}${sfxlist}.thickness.10.glmdir 3.0;
        done;
    done;
done
for sfxlist in ${temp_sfxlist};do
    for classlist in ${temp_classlist};do 
        for hemilist in ${temp_hemilist};do
            #echo freeview -f ${hemilist}.inflated:overlay=${hemilist}.${temp_title}${classlist}${sfxlist}.thickness.10.glmdir/${temp_mtx}/cache.th13.pos.sig.cluster.mgh:annot=${hemilist}.${temp_title}${classlist}${sfxlist}.thickness.10.glmdir/${temp_mtx}/cache.th13.pos.sig.ocn.annot -layout 1 -viewport 3d;
            freeview -f ${hemilist}.inflated:overlay=${hemilist}.${temp_title}${classlist}${sfxlist}.thickness.10.glmdir/${temp_mtx}/cache.th13.pos.sig.cluster.mgh:annot=${hemilist}.${temp_title}${classlist}${sfxlist}.thickness.10.glmdir/${temp_mtx}/cache.th13.pos.sig.ocn.annot -layout 1 -viewport 3d;
            #freeview -f ${hemilist}.inflated:overlay=${hemilist}.${temp_title}${classlist}${sfxlist}.thickness.10.glmdir/${temp_mtx}/cache.th30.pos.sig.cluster.mgh:annot=${hemilist}.${temp_title}${classlist}${sfxlist}.thickness.10.glmdir/${temp_mtx}/cache.th30.pos.sig.ocn.annot -layout 1 -viewport 3d;
        done;
    done;
done
"""

'\ncp *.mtx $SUBJECTS_DIR/fsaverage/surf\ncp *.fsgd $SUBJECTS_DIR/fsaverage/surf\ncd $SUBJECTS_DIR/fsaverage/surf\ntemp_fsgd=g2v1_\ntemp_title=group_age_\ntemp_sfxlist="_01 _02"\ntemp_classlist="AS AT AU"\ntemp_hemilist="lh rh"\ntemp_mtx=g2v1_common\n#temp_mtx=g2v1_gx_common\nfor sfxlist in ${temp_sfxlist};do\n    for classlist in ${temp_classlist};do \n        #echo fs_glm.sh ${temp_fsgd}${classlist}${sfxlist}.fsgd ${temp_mtx}.mtx;\n        yes| fs_glm.sh ${temp_fsgd}${classlist}${sfxlist}.fsgd ${temp_mtx}.mtx;\n    done;\ndone\nfor sfxlist in ${temp_sfxlist};do\n    for classlist in ${temp_classlist};do \n        for hemilist in ${temp_hemilist};do\n            #echo fs_view_glm.sh ${hemilist}.${temp_title}${classlist}${sfxlist}.thickness.10.glmdir ${temp_mtx}.mtx 1 4;\n            yes| fs_view_glm_allfwhm.sh ${hemilist}.${temp_title}${classlist}${sfxlist}.thickness.10.glmdir ${temp_mtx}.mtx 1 4;\n        done;\n    done;\ndone\n# 多重比較補正 CDT=1.3(p＝0.05)は不適。実際にはCDT=3.0(p=0.001)で行えば良い。